This sheet demonstrates basic exposure time calculation using CDR instrument values

In [1]:
import corgietc
import os
import json
import EXOSIMS.Prototypes.TargetList
import EXOSIMS.Prototypes.TimeKeeping
import copy
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# This loads the default input specification
scriptfile = os.path.join(os.environ["CGI_PERF_DIR"], "scripts", "cdrvals.json")
with open(scriptfile, "r") as f:
    specs = json.loads(f.read())

In [3]:
# This updates the input specification to create a target list with one target:
# HIP 53721 = 47 UMa
specs['fillPhotometry'] = True
specs["modules"]["StarCatalog"] = "HIPfromSimbad"
specs["catalogpath"] = [53721]

In [4]:
# Build the targetlist object
TL = EXOSIMS.Prototypes.TargetList.TargetList(**copy.deepcopy(specs))
OS = TL.OpticalSystem

/Users/ds264/Documents/gitrepos/EXOSIMS/EXOSIMS/Prototypes/TargetList.py:376: UserWarning: binary_filter is None but filterBinaries is True. Using binary_filter value of None.
  warnings.warn(


Imported HIPfromSimbad (implementation) from EXOSIMS.StarCatalog.HIPfromSimbad
Imported Nemati_2019 (implementation) from EXOSIMS.OpticalSystem.Nemati_2019
Imported Stark (implementation) from EXOSIMS.ZodiacalLight.Stark
Imported PostProcessing (prototype) from EXOSIMS.Prototypes.PostProcessing
Imported GalaxiesFaintStars (implementation) from EXOSIMS.BackgroundSources.GalaxiesFaintStars
Imported Completeness (prototype) from EXOSIMS.Prototypes.Completeness
Imported PlanetPopulation (prototype) from EXOSIMS.Prototypes.PlanetPopulation
Imported ForecasterMod (implementation) from EXOSIMS.PlanetPhysicalModel.ForecasterMod
1 targets imported from star catalog.
1 targets remain after removing those where spectral class cannot be established.
1 targets remain after removing white dwarfs and subdwarfs
1 targets remain after nan filtering.
1 targets remain after removing zero luminosity targets.
Loaded star fluxes values from /Users/ds264/.EXOSIMS/cache/TargetList_HIPfromSimbad_nStars_1_mode_

/Users/ds264/Documents/gitrepos/EXOSIMS/EXOSIMS/Prototypes/OpticalSystem.py:619: UserWarning: Input focal length (0.26 m) does not match value from pixelScale (134.07 m) for instrument Conservative_Amici_Spec. This feels like a mistkae.
  warnings.warn(
/Users/ds264/Documents/gitrepos/EXOSIMS/EXOSIMS/Prototypes/OpticalSystem.py:619: UserWarning: Input focal length (0.26 m) does not match value from pixelScale (134.07 m) for instrument Optimistic_Amici_Spec. This feels like a mistkae.
  warnings.warn(
/Users/ds264/Documents/gitrepos/EXOSIMS/EXOSIMS/Prototypes/OpticalSystem.py:1410: UserWarning: core_mean_intensity has larger IWA than current system value for EB_HLC-565. Updating to match table, but you should check your inputs.
  warnings.warn(
/Users/ds264/Documents/gitrepos/EXOSIMS/EXOSIMS/Prototypes/OpticalSystem.py:1410: UserWarning: core_mean_intensity has larger IWA than current system value for DRM_HLC-565. Updating to match table, but you should check your inputs.
  warnings.war

In [5]:
# let's check to make sure we have the right target
print(f"Name: {TL.Name}")
print(f"is located at {TL.coords}")
print(f"and has a V band magnitude of {TL.Vmag}")

Name: ['HIP 53721']
is located at <SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    [(164.86655313, 40.43025571, 13.88753871)]>
and has a V band magnitude of [5.15719986]


In [6]:
# create a Timekeeping object and advance the mission time a bit
TK = EXOSIMS.Prototypes.TimeKeeping.TimeKeeping(missionLife = 5.25)   # 63 months in years is 5.25, 21 months is 1.75
TK.allocate_time(21*30.4375*u.d);

OBendTimes is: [1917.5625] d


In [7]:
# Grab the band 1 observing mode
mode = list(filter(lambda mode: mode['instName'] == 'Optimistic_NF_Imager', OS.observingModes))[0] # Band 1
mode

{'instName': 'Optimistic_NF_Imager',
 'systName': 'DRM_HLC-565',
 'detectionMode': True,
 'ContrastScenario': '2019_PDR_Update',
 'SNR': 5.0,
 'tau_pol': 1,
 'GCRFlux': 5,
 'photons_per_relativistic_event': 250,
 'luminescingOpticalArea': 0.7854,
 'OpticalThickness': 4,
 'luminescingOpticalDistance': 0.1,
 's_baffling': 0.001,
 'timeMultiplier': 1.0,
 'inst': {'name': 'Optimistic_NF_Imager',
  'QE': <function EXOSIMS.Prototypes.OpticalSystem.OpticalSystem.populate_scienceInstruments.<locals>.<lambda>(l)>,
  'FoV': <Quantity 9.5 arcsec>,
  'pixelNumber': 1024.0,
  'pixelSize': <Quantity 1.3e-05 m>,
  'sread': 1.7e-06,
  'kRN': 75,
  'idark': <Quantity 3.e-05 1 / s>,
  'CIC': 0.0021,
  'CTE_derate': 0.93,
  'dark_derate': 1,
  'refl_derate': 1,
  'MUF_thruput': 0.909,
  'lam_d': 575.855987,
  'lam_c': 508.0,
  'k_samp': 0.2,
  'texp': <Quantity 3. s>,
  'ENF': 1.0,
  'THPUT': '$CGI_PERF_DIR/CDR/Photometry/THPT_EB_NFB1_200730.csv',
  'DET': '$CGI_PERF_DIR/CDR/Photometry/DET_CBE_210127.csv

In [8]:
# set values necessary for integration time calculation
(sInds, fZ, dMags, WAs) = ([0], #index of star in target list
                        TL.ZodiacalLight.fZ0, #local zodiacal light
                        np.array([20]), #dMag limit to integrate to
                        (3.6*u.AU/TL.dist[0]).to(u.mas,equivalencies=u.dimensionless_angles()), #WA of object
                        )
fEZ = TL.ZodiacalLight.fEZ(TL.MV[sInds], [90]*u.deg, 3.6*u.AU ) # exozodiacal light

NameError: name 'sInds' is not defined

In [ ]:
intTimes = OS.calc_intTime(TL, sInds, fZ, fEZ, dMags, WAs, mode, TK=TK)
intTimes

In [ ]:
# now let's try a range of delta-mags
dMags = np.linspace(19, 23, 100)
intTimes = OS.calc_intTime(TL, sInds, fZ, fEZ, dMags, WAs, mode, TK=TK)

In [ ]:
plt.figure()
plt.semilogy(dMags, intTimes)
plt.xlabel(r"Target $\Delta$mag")
plt.ylabel(f"Integration Time ({intTimes.unit})")
plt.title(fr"Maximum Achievable $\Delta$mag = {np.min(dMags[np.isnan(intTimes)]) :.2f}");